### Import

In [10]:
!pip install optuna-integration[sklearn]

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder

import optuna
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from optuna.integration import OptunaSearchCV

### Data Load

In [2]:
train = pd.read_csv('./Data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./Data/test.csv').drop(columns=['ID'])

In [3]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [4]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [5]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [6]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [7]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [8]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_encoded, y, test_size=0.2, random_state=42, stratify=y
)

### Train

Stacking model

In [ ]:
# Optuna 최적화 함수 정의
def objective(trial):
    # RandomForest 하이퍼파라미터 튜닝
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 50, 300)
    rf_max_depth = trial.suggest_int("rf_max_depth", 3, 20)
    
    # XGBoost 하이퍼파라미터 튜닝
    xgb_n_estimators = trial.suggest_int("xgb_n_estimators", 50, 300)
    xgb_max_depth = trial.suggest_int("xgb_max_depth", 3, 20)
    xgb_learning_rate = trial.suggest_float("xgb_learning_rate", 0.01, 0.3, log=True)
    
    # Logistic Regression 하이퍼파라미터 튜닝
    lr_C = trial.suggest_float("lr_C", 0.01, 10, log=True)

    # 베이스 모델 정의
    base_models = [
        ('rf', RandomForestClassifier(n_estimators=rf_n_estimators, max_depth=rf_max_depth, random_state=42, n_jobs=-1)),
        ('xgb', XGBClassifier(n_estimators=xgb_n_estimators, max_depth=xgb_max_depth, learning_rate=xgb_learning_rate, eval_metric='logloss', random_state=42, n_jobs=-1)),
        ('lr', LogisticRegression(C=lr_C, max_iter=10000, random_state=42))
    ]

    # 메타 모델 정의
    meta_model = LogisticRegression(max_iter=10000, random_state=42)

    # 스태킹 모델 구성
    stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=StratifiedKFold(n_splits=3))

    # AUC 기반 교차 검증 수행
    auc_scores = cross_val_score(stacking_clf, X_train, y_train, cv=StratifiedKFold(n_splits=3), scoring='roc_auc', n_jobs=-1)
    
    return np.mean(auc_scores)

In [11]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar = True)

[I 2025-02-20 00:22:03,388] A new study created in memory with name: no-name-8a995d77-c2c8-491a-9d14-a114db151687


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-02-20 00:23:19,205] Trial 0 finished with value: 0.7337700293167558 and parameters: {'rf_n_estimators': 83, 'rf_max_depth': 18, 'xgb_n_estimators': 107, 'xgb_max_depth': 15, 'xgb_learning_rate': 0.26234152553267853, 'lr_C': 0.015535740219361202}. Best is trial 0 with value: 0.7337700293167558.
[I 2025-02-20 00:25:54,745] Trial 1 finished with value: 0.735816546711944 and parameters: {'rf_n_estimators': 136, 'rf_max_depth': 15, 'xgb_n_estimators': 245, 'xgb_max_depth': 17, 'xgb_learning_rate': 0.015196943130613295, 'lr_C': 7.35194038417909}. Best is trial 1 with value: 0.735816546711944.
[I 2025-02-20 00:27:52,138] Trial 2 finished with value: 0.7342434987709842 and parameters: {'rf_n_estimators': 223, 'rf_max_depth': 19, 'xgb_n_estimators': 196, 'xgb_max_depth': 19, 'xgb_learning_rate': 0.055470737198455654, 'lr_C': 0.023825391852176955}. Best is trial 1 with value: 0.735816546711944.
[I 2025-02-20 00:30:28,464] Trial 3 finished with value: 0.735333260832361 and parameters: {'r

In [12]:
# 최적 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)

Best hyperparameters: {'rf_n_estimators': 254, 'rf_max_depth': 18, 'xgb_n_estimators': 244, 'xgb_max_depth': 4, 'xgb_learning_rate': 0.06548201380816693, 'lr_C': 2.1656133988420163}


In [13]:
# 최적 하이퍼파라미터로 모델 학습
best_params = study.best_params
rf_best = RandomForestClassifier(n_estimators=best_params["rf_n_estimators"], max_depth=best_params["rf_max_depth"], random_state=42)
xgb_best = XGBClassifier(n_estimators=best_params["xgb_n_estimators"], max_depth=best_params["xgb_max_depth"], learning_rate=best_params["xgb_learning_rate"], use_label_encoder=False, eval_metric='logloss', random_state=42)
lr_best = LogisticRegression(C=best_params["lr_C"], max_iter=1000, random_state=42)

# 최적화된 스태킹 모델 생성
model = StackingClassifier(estimators=[('rf', rf_best), ('xgb', xgb_best), ('lr', lr_best)], final_estimator=LogisticRegression(max_iter=1000, random_state=42), cv=StratifiedKFold(n_splits=5))

In [15]:
model.fit(X_train_encoded, y)

c:\Users\vhehr\.conda\envs\LGAimers6\Lib\site-packages\xgboost\core.py:158: UserWarning: [03:05:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\vhehr\.conda\envs\LGAimers6\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\vhehr\.conda\envs\LGAimers6\Lib\site-packages\xgboost\core.py:158: UserWarning: [03:09:37] WARNING: C:\buildkite-agent\builds\buildkit

StackingClassifier(cv=StratifiedKFold(n_splits=5,
        random_state=RandomState(MT19937) at 0x26CC57DBC40, shuffle=False),
                   estimators=[('rf',
                                RandomForestClassifier(max_depth=18,
                                                       n_estimators=254,
                                                       random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              earl...
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=4,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=244, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...)),
                               ('lr',
                                LogisticRegression(C=2.1656133988420163,
                                                   max_iter=1000,
                                                   random_state=42))],
                   final_estimator=LogisticRegression(max_iter=1000,
                                                      random_state=42))

In [16]:
from sklearn.metrics import accuracy_score, roc_auc_score

# 예측
y_train_pred = model.predict(X_train_encoded)
y_train_proba = model.predict_proba(X_train_encoded)[:, 1]  # ROC-AUC Score 계산용

# 평가
accuracy = accuracy_score(y, y_train_pred)
roc_auc = roc_auc_score(y, y_train_proba)

# 출력
print(f"Model - Accuracy: {accuracy:.4f}")
print(f"Model - ROC-AUC Score: {roc_auc:.4f}")

Model - Accuracy: 0.7632
Model - ROC-AUC Score: 0.7785


### Predict

In [17]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

### Model Save

In [19]:
import joblib

# 모델 저장
joblib.dump(model, './stacking_optimization_model.pkl')

# # 저장한 모델 불러오기
# loaded_model = joblib.load('./stacking_model.pkl')


['./stacking_optimization_model.pkl']

### Submission

In [20]:
sample_submission = pd.read_csv('./Data/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [21]:
sample_submission.to_csv('./Stacking_optimization_submit.csv', index=False)